In [1]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'  
from pyspark import SparkContext  
from pyspark.streaming import StreamingContext  
from pyspark.streaming.kafka import KafkaUtils   

batch_interval = 1
window_length = 12 * batch_interval
frequency = 6 * batch_interval

sc = SparkContext(appName="CTR")
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, batch_interval)

directKafkaStream = KafkaUtils.createDirectStream(ssc, ['topic_click'], {"metadata.broker.list": 'localhost:9092'})

In [2]:
lines = directKafkaStream.map(lambda x: x[1])

In [ ]:
import re
def mk_int(s):
    s = s.strip()
    return int(s) if s else 0
def tuple_sum(a, b):
    return [a[0]+b[0], a[1]+b[1]]

In [ ]:
ctr = lines.flatMap(lambda line: [line.split("\t")[-1]]) \
    .map(lambda click: ("ctr", [mk_int(click), 1])) \
    .reduceByKeyAndWindow(lambda a, b: tuple_sum(a, b), None, 12, 6) \
    .flatMap(lambda row: ("ctr = ", 100.0 * row[1][0] / row[1][1]))
ctr.pprint()

ssc.start()  
ssc.awaitTermination()

-------------------------------------------
Time: 2017-09-19 22:52:59
-------------------------------------------
ctr = 
0.299428364032

-------------------------------------------
Time: 2017-09-19 22:53:05
-------------------------------------------
ctr = 
0.316789862724

-------------------------------------------
Time: 2017-09-19 22:53:11
-------------------------------------------
ctr = 
0.365448504983

-------------------------------------------
Time: 2017-09-19 22:53:17
-------------------------------------------
ctr = 
0.333364200389

-------------------------------------------
Time: 2017-09-19 22:53:23
-------------------------------------------
ctr = 
0.314867424242

-------------------------------------------
Time: 2017-09-19 22:53:29
-------------------------------------------
ctr = 
0.345423143351

-------------------------------------------
Time: 2017-09-19 22:53:35
-------------------------------------------
ctr = 
0.331925165963

-----------------------------------------

In [ ]:
ssc.stop()
sc.stop()